# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/4. normalized-50/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (604, 26)
y_train: (604, 2)
x_test: (604, 26)
y_test: (604, 2)


# Load autoencoder 1

In [3]:
autoencoder_path = 'results/34-autoencoder-1-01/model/2018-03-09-03-01-07/weights.hdf5'
weights = load_model(autoencoder_path)
weights.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 26)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                270       
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 26)                286       
Total params: 556
Trainable params: 556
Non-trainable params: 0
_________________________________________________________________


# Copy autoencoder's weights except the last layer

In [4]:
x = Input(shape=(x_train.shape[1],))
y = Dense(units = 10, activation='relu')(x)

auto_1 = Model(x, y)

for i in range(2):
    auto_1.layers[i].set_weights(weights.layers[i].get_weights())
    
x_train = auto_1.predict(x_train)
x_test = auto_1.predict(x_test)
print(x_train.shape)
print(x_test.shape)

(604, 10)
(604, 10)


# Tunable parameters

In [5]:
epochs = 3000
batch_size = 20
learning_rate = 0.0001
decay = 0.0001
loss = keras.losses.categorical_crossentropy

# Autoencoder #2

In [6]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/37-autoencoder-2-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
def create_model(layer_size, dropout):
    global date
    x = Input(shape=(x_train.shape[1],))

    y = Dense(units = layer_size, activation='relu')(x)
    y = Dropout(dropout)(y)

    y = Dense(units = x_train.shape[1], activation='sigmoid')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Hidden layer units': layer_size,
        'Dropout': dropout,
        'Learning Rate': learning_rate,
        'Decay': decay,
        'Epochs': epochs,
        'Batch Size': batch_size,
        'Loss': loss
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=loss, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [7]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, **kwargs)

# Grid search

In [8]:
model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

layers = [8, 7, 6]
dropouts = [0, 0.4]
param_grid = dict(
    layer_size=layers, 
    dropout=dropouts
)

grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test, x_test)))
grid_result = grid.fit(x_train, x_train)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[CV] dropout=0, layer_size=8 .........................................
Train on 604 samples, validate on 604 samples
Epoch 1/3000
604/604 [==============================] - 0s - loss: 28.4760 - acc: 0.1275 - val_loss: 28.1532 - val_acc: 0.1126
Epoch 2/3000
604/604 [==============================] - 0s - loss: 28.2793 - acc: 0.1225 - val_loss: 27.9643 - val_acc: 0.1175
Epoch 3/3000
604/604 [==============================] - 0s - loss: 28.0860 - acc: 0.1225 - val_loss: 27.7765 - val_acc: 0.1175
Epoch 4/3000
604/604 [==============================] - 0s - loss: 27.8957 - acc: 0.1275 - val_loss: 27.5941 - val_acc: 0.1192
Epoch 5/3000
604/604 [==============================] - 0s - loss: 27.7094 - acc: 0.1308 - val_loss: 27.4112 - val_acc: 0.1225
Epoch 6/3000
604/604 [==============================] - 0s - loss: 27.5231 - acc: 0.1358 - val_loss: 27.2310 - val_acc: 0.1291
Epoch 7/3000
604/604 [==============================] - 0s - l

604/604 [==============================] - 0s - loss: 22.1718 - acc: 0.4371 - val_loss: 22.0782 - val_acc: 0.4586
Epoch 64/3000
604/604 [==============================] - 0s - loss: 22.1458 - acc: 0.4421 - val_loss: 22.0534 - val_acc: 0.4619
Epoch 65/3000
604/604 [==============================] - 0s - loss: 22.1209 - acc: 0.4454 - val_loss: 22.0296 - val_acc: 0.4669
Epoch 66/3000
604/604 [==============================] - 0s - loss: 22.0970 - acc: 0.4536 - val_loss: 22.0060 - val_acc: 0.4685
Epoch 67/3000
604/604 [==============================] - 0s - loss: 22.0736 - acc: 0.4536 - val_loss: 21.9841 - val_acc: 0.4702
Epoch 68/3000
604/604 [==============================] - 0s - loss: 22.0516 - acc: 0.4536 - val_loss: 21.9631 - val_acc: 0.4719
Epoch 69/3000
604/604 [==============================] - 0s - loss: 22.0305 - acc: 0.4553 - val_loss: 21.9427 - val_acc: 0.4785
Epoch 70/3000
604/604 [==============================] - 0s - loss: 22.0100 - acc: 0.4586 - val_loss: 21.9233 - val_ac

604/604 [==============================] - 0s - loss: 21.4766 - acc: 0.5149 - val_loss: 21.4110 - val_acc: 0.5414
Epoch 127/3000
604/604 [==============================] - 0s - loss: 21.4717 - acc: 0.5232 - val_loss: 21.4064 - val_acc: 0.5430
Epoch 128/3000
604/604 [==============================] - 0s - loss: 21.4669 - acc: 0.5281 - val_loss: 21.4018 - val_acc: 0.5464
Epoch 129/3000
604/604 [==============================] - 0s - loss: 21.4624 - acc: 0.5315 - val_loss: 21.3975 - val_acc: 0.5480
Epoch 130/3000
604/604 [==============================] - 0s - loss: 21.4578 - acc: 0.5331 - val_loss: 21.3932 - val_acc: 0.5563
Epoch 131/3000
604/604 [==============================] - 0s - loss: 21.4534 - acc: 0.5364 - val_loss: 21.3890 - val_acc: 0.5613
Epoch 132/3000
604/604 [==============================] - 0s - loss: 21.4490 - acc: 0.5430 - val_loss: 21.3849 - val_acc: 0.5646
Epoch 133/3000
604/604 [==============================] - 0s - loss: 21.4447 - acc: 0.5447 - val_loss: 21.3807 -

604/604 [==============================] - 0s - loss: 21.2393 - acc: 0.6258 - val_loss: 21.1870 - val_acc: 0.6242
Epoch 190/3000
604/604 [==============================] - 0s - loss: 21.2354 - acc: 0.6242 - val_loss: 21.1834 - val_acc: 0.6242
Epoch 191/3000
604/604 [==============================] - 0s - loss: 21.2314 - acc: 0.6258 - val_loss: 21.1795 - val_acc: 0.6242
Epoch 192/3000
604/604 [==============================] - 0s - loss: 21.2275 - acc: 0.6291 - val_loss: 21.1759 - val_acc: 0.6275
Epoch 193/3000
604/604 [==============================] - 0s - loss: 21.2234 - acc: 0.6341 - val_loss: 21.1721 - val_acc: 0.6209
Epoch 194/3000
604/604 [==============================] - 0s - loss: 21.2193 - acc: 0.6308 - val_loss: 21.1683 - val_acc: 0.6192
Epoch 195/3000
604/604 [==============================] - 0s - loss: 21.2154 - acc: 0.6291 - val_loss: 21.1645 - val_acc: 0.6159
Epoch 196/3000
604/604 [==============================] - 0s - loss: 21.2113 - acc: 0.6325 - val_loss: 21.1608 -

604/604 [==============================] - 0s - loss: 21.0397 - acc: 0.6589 - val_loss: 21.0017 - val_acc: 0.6407
Epoch 253/3000
604/604 [==============================] - 0s - loss: 21.0378 - acc: 0.6606 - val_loss: 21.0000 - val_acc: 0.6490
Epoch 254/3000
604/604 [==============================] - 0s - loss: 21.0359 - acc: 0.6606 - val_loss: 20.9982 - val_acc: 0.6474
Epoch 255/3000
604/604 [==============================] - 0s - loss: 21.0341 - acc: 0.6606 - val_loss: 20.9964 - val_acc: 0.6507
Epoch 256/3000
604/604 [==============================] - 0s - loss: 21.0323 - acc: 0.6606 - val_loss: 20.9947 - val_acc: 0.6523
Epoch 257/3000
604/604 [==============================] - 0s - loss: 21.0306 - acc: 0.6606 - val_loss: 20.9929 - val_acc: 0.6507
Epoch 258/3000
604/604 [==============================] - 0s - loss: 21.0288 - acc: 0.6639 - val_loss: 20.9913 - val_acc: 0.6507
Epoch 259/3000
604/604 [==============================] - 0s - loss: 21.0271 - acc: 0.6656 - val_loss: 20.9898 -

...

604/604 [==============================] - 0s - loss: 21.4447 - acc: 0.5397 - val_loss: 21.2222 - val_acc: 0.5993
Epoch 2037/3000
604/604 [==============================] - 0s - loss: 21.4151 - acc: 0.5381 - val_loss: 21.2222 - val_acc: 0.5993
Epoch 2038/3000
604/604 [==============================] - 0s - loss: 21.4327 - acc: 0.5646 - val_loss: 21.2221 - val_acc: 0.6010
Epoch 2039/3000
604/604 [==============================] - 0s - loss: 21.4424 - acc: 0.5132 - val_loss: 21.2221 - val_acc: 0.6010
Epoch 2040/3000
604/604 [==============================] - 0s - loss: 21.4489 - acc: 0.5099 - val_loss: 21.2220 - val_acc: 0.6010
Epoch 2041/3000
604/604 [==============================] - 0s - loss: 21.4214 - acc: 0.5381 - val_loss: 21.2220 - val_acc: 0.6043
Epoch 2042/3000
604/604 [==============================] - 0s - loss: 21.4593 - acc: 0.5298 - val_loss: 21.2220 - val_acc: 0.6043
Epoch 2043/3000
604/604 [==============================] - 0s - loss: 21.4675 - acc: 0.5546 - val_loss: 21

604/604 [==============================] - 0s - loss: 21.4629 - acc: 0.5480 - val_loss: 21.2200 - val_acc: 0.6010
Epoch 2100/3000
604/604 [==============================] - 0s - loss: 21.4555 - acc: 0.5066 - val_loss: 21.2200 - val_acc: 0.6010
Epoch 2101/3000
604/604 [==============================] - 0s - loss: 21.4485 - acc: 0.5298 - val_loss: 21.2199 - val_acc: 0.6010
Epoch 2102/3000
604/604 [==============================] - 0s - loss: 21.4518 - acc: 0.5513 - val_loss: 21.2198 - val_acc: 0.6010
Epoch 2103/3000
604/604 [==============================] - 0s - loss: 21.4158 - acc: 0.5679 - val_loss: 21.2197 - val_acc: 0.6010
Epoch 2104/3000
604/604 [==============================] - 0s - loss: 21.4558 - acc: 0.5331 - val_loss: 21.2196 - val_acc: 0.6010
Epoch 2105/3000
604/604 [==============================] - 0s - loss: 21.4574 - acc: 0.5397 - val_loss: 21.2196 - val_acc: 0.6010
Epoch 2106/3000
604/604 [==============================] - 0s - loss: 21.4475 - acc: 0.5298 - val_loss: 21

604/604 [==============================] - 0s - loss: 21.4521 - acc: 0.5265 - val_loss: 21.2178 - val_acc: 0.6043
Epoch 2163/3000
604/604 [==============================] - 0s - loss: 21.4116 - acc: 0.5447 - val_loss: 21.2178 - val_acc: 0.6043
Epoch 2164/3000
604/604 [==============================] - 0s - loss: 21.4629 - acc: 0.5464 - val_loss: 21.2177 - val_acc: 0.6043
Epoch 2165/3000
604/604 [==============================] - 0s - loss: 21.4296 - acc: 0.5629 - val_loss: 21.2177 - val_acc: 0.6043
Epoch 2166/3000
604/604 [==============================] - 0s - loss: 21.4495 - acc: 0.5430 - val_loss: 21.2177 - val_acc: 0.6043
Epoch 2167/3000
604/604 [==============================] - 0s - loss: 21.4414 - acc: 0.5530 - val_loss: 21.2176 - val_acc: 0.6043
Epoch 2168/3000
604/604 [==============================] - 0s - loss: 21.4594 - acc: 0.5397 - val_loss: 21.2176 - val_acc: 0.6043
Epoch 2169/3000
604/604 [==============================] - 0s - loss: 21.4431 - acc: 0.5530 - val_loss: 21

604/604 [==============================] - 0s - loss: 21.4133 - acc: 0.5513 - val_loss: 21.2157 - val_acc: 0.6076
Epoch 2226/3000
604/604 [==============================] - 0s - loss: 21.4726 - acc: 0.5497 - val_loss: 21.2157 - val_acc: 0.6093
Epoch 2227/3000
604/604 [==============================] - 0s - loss: 21.4314 - acc: 0.5083 - val_loss: 21.2157 - val_acc: 0.6093
Epoch 2228/3000
604/604 [==============================] - 0s - loss: 21.4361 - acc: 0.5464 - val_loss: 21.2156 - val_acc: 0.6093
Epoch 2229/3000
604/604 [==============================] - 0s - loss: 21.4217 - acc: 0.5381 - val_loss: 21.2155 - val_acc: 0.6093
Epoch 2230/3000
604/604 [==============================] - 0s - loss: 21.4511 - acc: 0.5215 - val_loss: 21.2155 - val_acc: 0.6093
Epoch 2231/3000
604/604 [==============================] - 0s - loss: 21.4263 - acc: 0.5513 - val_loss: 21.2155 - val_acc: 0.6093
Epoch 2232/3000
604/604 [==============================] - 0s - loss: 21.4702 - acc: 0.5331 - val_loss: 21

KeyboardInterrupt: 